In [ ]:
## library
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

# Data Calculation

## Total Calories

In [ ]:
def calorie(gender, weight, height, age, activities):
  #for male
  if gender == 0:
    cal = 66+(13.7*weight)+(5*height)-(6.8*age)
  #for female
  elif gender == 1:
    cal = 655+(9.6*weight)+(1.8*height)-(4.7*age)
  #activities
  ##sgt jarang olahraga
  if activities == 1:
    tot_cal = cal*1.2
  ##jarang olahraga
  elif activities == 2:
    tot_cal = cal*1.375
  ##cukup olahraga
  elif activities == 3:
    tot_cal = cal*1.55
  ##sering olahraga
  elif activities == 4:
    tot_cal = cal*1.725
  ##sangat sering olahraga
  elif activities == 5:
    tot_cal = cal*1.9
  return round(tot_cal,2)

## Calorie each meal

In [ ]:
def cal_each_meal(tot_cal, tot_meals):
  # meals: breakfast, lunch, dinner
  if tot_meals == 3:
    breakfast_cal = round(tot_cal*0.3,2)
    lunch_cal = round(tot_cal*0.4,2)
    dinner_cal = round(tot_cal*0.3,2)
  # meals: breakfast, lunch, dinner, snack
  #elif tot_meals == 4:
    #breakfast_cal = round(tot_cal*0.25,2)
    #lunch_cal = round(tot_cal*0.4,2)
    #dinner_cal = round(tot_cal*0.25,2)
    #snack_cal = round(tot_cal*0.1,2)
  return breakfast_cal, lunch_cal, dinner_cal

## Nutrition needs each meal

In [ ]:
def nutrition_needs(breakfast, lunch, dinner):
  meals = [breakfast, lunch, dinner]
  carbohydrates = []
  fats = []
  proteins = []
  for meal in meals:
    carbohydrate = round((meal*0.65)/4,2) #dibagi 4, calorie ke gram
    carbohydrates.append(carbohydrate)
    fat = round((meal*0.2)/9,2) #dibagi 9, calorie ke gram
    fats.append(fat)
    protein = round((meal*0.15)/4,2) #dibagi 4, calorie ke gram
    proteins.append(protein)
  ## making dataframe
  y_train = pd.DataFrame(meals, columns=['calorie'])
  user_train = pd.DataFrame({'carbohydrate':carbohydrates,
                             'fat':fats,
                             'protein':proteins})
  return y_train, user_train

# Data Collection

In [ ]:
# syntax import data
## data user --> y_train dan user_train
## data food --> food_train

## calculation from the data user

In [ ]:
tot_cal = calorie(gender, weight, height, age, activities)
breakfast, lunch, dinner, snack = cal_each_meal(tot_cal, tot_meals)
y_train, user_train = nutrition_needs(breakfast, lunch, dinner, snack)

 # Preparing Data

## Scaling

In [ ]:
# scale training data
item_train_unscaled = food_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(food_train)
item_train = scalerItem.transform(food_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))

## Spliting

In [ ]:
# split to 0.8 and 0.2
food_train, food_test = train_test_split(food_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)

# Neural Network; Content-based filtering

## Model

In [ ]:
num_outputs = 10
num_user_features = user_train.shape[1]
num_item_features = food_train.shape[1]

tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###
  tf.keras.layers.Dense(256,activation='relu'),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(num_outputs)
    ### END CODE HERE ###
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###
  tf.keras.layers.Dense(256,activation='relu'),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(num_outputs)
    ### END CODE HERE ###
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

## Compile & Fit

In [ ]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [ ]:
tf.random.set_seed(1)
model.fit([user_train, food_train], y_train, epochs=30)

In [ ]:
model.evaluate([user_test, food_test], y_test)